In [1]:
import os
import gc
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import timedelta


import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from gensim.models import Word2Vec

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)


trans_info = pd.read_csv('账户交易信息.csv')
static_info = pd.read_csv('账户静态信息.csv')
session = pd.read_csv('./session.csv')

train_set = pd.read_csv('训练集标签.csv')
test_set = pd.read_csv('test_dataset.csv')

/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
%%time
# 合并账户静态信息
static_info['khrq']  = pd.to_datetime(static_info['khrq'], format='%Y-%m-%d')
static_info['year']  = static_info['khrq'].dt.year
static_info['month'] = static_info['khrq'].dt.month
static_info['day']   = static_info['khrq'].dt.day

# 自然数编码
def label_encode(series):
    unique = list(series.unique())
    return series.map(dict(zip(
        unique, range(series.nunique())
    )))

for col in ['khjgdh']:
    static_info[col] = label_encode(static_info[col])
    
static_info['age_level'] = static_info['年龄'] // 5
keep_cols = ['zhdh','year','month','day','khjgdh','xb','年龄', 'khrq', 'age_level']

train_set = train_set.merge(static_info[keep_cols], on=['zhdh'], how='left')
test_set  = test_set.merge(static_info[keep_cols], on=['zhdh'], how='left')

basic_fea = list(train_set.columns)
print(train_set.columns)

Index(['zhdh', 'black_flag', 'year', 'month', 'day', 'khjgdh', 'xb', '年龄',
       'khrq', 'age_level'],
      dtype='object')
CPU times: user 18.9 ms, sys: 2.32 ms, total: 21.2 ms
Wall time: 24 ms


In [3]:
%%time
# 静态时间特征
def basic_time_fea(df):
    col = 'date'
    df[col] = pd.to_datetime(df['jyrq'] + ' ' + df['jysj'])
    df['jyrq_year'] = df[col].dt.year
    df['jyrq_month'] = df[col].dt.month
    df['jyrq_day'] = df[col].dt.day
    df['jyrq_hour'] = df[col].dt.hour
    
    df['jyrq_weekofyear'] = df[col].dt.weekofyear
    df['jyrq_dayofyear'] = df[col].dt.dayofyear
    

basic_time_fea(trans_info)
trans_info = trans_info.sort_values(by='date')

col = ['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq','jysj', 'jyqd', 'zydh', 'dfmccd']
trans_info = trans_info.merge(session[col + ['session_id']], how='left', on=col)
print('time col - ', session.columns)

time col -  Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd', 'date', 'session_id'],
      dtype='object')
CPU times: user 3.42 s, sys: 220 ms, total: 3.64 s
Wall time: 3.65 s


In [4]:
%%time
# 转入转出次数、金额比例、dfzh个数
in_count = trans_info[trans_info['jdbj'] == 1].groupby('zhdh').size().reset_index().rename(columns={0: 'in_count'})
out_count = trans_info[trans_info['jdbj'] == 0].groupby('zhdh').size().reset_index().rename(columns={0: 'out_count'})

in_sum = trans_info[trans_info['jdbj'] == 1].groupby('zhdh')['jyje'].sum().reset_index().rename(columns={'jyje': 'in_sum'})
out_sum = trans_info[trans_info['jdbj'] == 0].groupby('zhdh')['jyje'].sum().reset_index().rename(columns={'jyje': 'out_sum'})

in_uniq = trans_info[trans_info['jdbj'] == 1].groupby('zhdh')['dfzh'].\
                            nunique().reset_index().rename(columns={'dfzh': 'in_uniq'})
out_uniq = trans_info[trans_info['jdbj'] == 0].groupby('zhdh')['dfzh'].\
                            nunique().reset_index().rename(columns={'dfzh': 'out_uniq'})


in_abs_uniq = trans_info[trans_info['jdbj'] == 1].groupby('zhdh')['zydh'].\
                            nunique().reset_index().rename(columns={'zydh': 'in_abs_uniq'})
out_abs_uniq = trans_info[trans_info['jdbj'] == 0].groupby('zhdh')['zydh'].\
                            nunique().reset_index().rename(columns={'zydh': 'out_abs_uniq'})


for tmp in [in_count, out_count, in_sum, out_sum, in_uniq, out_uniq, in_abs_uniq, out_abs_uniq]:
    train_set = train_set.merge(tmp, how='left', on='zhdh')
    test_set = test_set.merge(tmp, how='left', on='zhdh')
    
train_set['trans_count_ration'] = train_set['in_count'] / train_set['out_count']
test_set['trans_count_ration'] = test_set['in_count'] / test_set['out_count']

train_set['trans_sum_ration'] = train_set['in_sum'] / train_set['out_sum']
test_set['trans_sum_ration'] = test_set['in_sum'] / test_set['out_sum']

CPU times: user 1.32 s, sys: 78.3 ms, total: 1.4 s
Wall time: 1.41 s


In [5]:
%%time
# 每小时交易/转出超过3，5，8，10 次的用户
trans_out = trans_info[trans_info['jdbj'] == 0].groupby(['zhdh', 'jyrq_dayofyear', 'jyrq_hour']).size().reset_index()
trans_all = trans_info.groupby(['zhdh', 'jyrq_dayofyear', 'jyrq_hour']).size().reset_index()

for num in [3, 5, 8, 10]:
    tmp1 = trans_out[trans_out[0] > num]['zhdh'].unique()
    tmp2 = trans_all[trans_all[0] > num]['zhdh'].unique()
    
    train_set['trans_out_count_{}'.format(num)], test_set['trans_out_count_{}'.format(num)] = 0, 0
    train_set.loc[train_set['zhdh'].isin(tmp1), 'trans_out_count_{}'.format(num)] = 1
    test_set.loc[test_set['zhdh'].isin(tmp1), 'trans_out_count_{}'.format(num)] = 1

    train_set['trans_all_count_{}'.format(num)], test_set['trans_all_count_{}'.format(num)] = 0, 0
    train_set.loc[train_set['zhdh'].isin(tmp2), 'trans_all_count_{}'.format(num)] = 1
    test_set.loc[test_set['zhdh'].isin(tmp2), 'trans_all_count_{}'.format(num)] = 1

CPU times: user 480 ms, sys: 7.33 ms, total: 488 ms
Wall time: 486 ms


In [6]:
%%time
# 转入转出抵消
trans_info.loc[trans_info['jdbj'] == 0, 'jdbj'] = -1
trans_info['balance'] = trans_info['jdbj'] * trans_info['jyje']

def count_zero(x):
    count = 0
    tag = 0
    record = []
    for i in x:
        if i > 0 and tag == 0:
            tag += 1
        if i < 0 and tag == 1:
            tag += 1
            
        if tag == 2 and i > 0:

            if sum(record) == 0:
                count += 1
            tag = 0
            record = []
        record.append(i)
    return count
        

user_info = trans_info.groupby('zhdh')['balance'].apply(list).reset_index()
user_info['zero_count'] = user_info['balance'].map(count_zero)
del user_info['balance']

train_set = train_set.merge(user_info, how='left', on='zhdh')
test_set = test_set.merge(user_info, how='left', on='zhdh')

CPU times: user 517 ms, sys: 17.4 ms, total: 535 ms
Wall time: 534 ms


In [7]:
%%time
# word2vec 特征
tmp = trans_info.groupby('zhdh')['balance'].apply(list).reset_index()
tmp['sent'] = tmp['balance'].apply(lambda x: [str(i) for i in x])

sentence = []
for line in tmp['sent'].tolist():
    sentence.append([str(float(l)) for idx, l in enumerate(line)])

vs = 10
model = Word2Vec(sentence, vector_size=vs, window=2, min_count=1, workers=7)
new_col = ['word2vec_{}'.format(i) for i in range(vs)]
for i in range(vs):
    tmp['word2vec_{}'.format(i)] = tmp['balance'].apply(lambda x: sum([model.wv[str(i)] for i in x])[i])


train_set = train_set.merge(tmp[['zhdh'] + new_col], how='left', on='zhdh')
test_set = test_set.merge(tmp[['zhdh'] + new_col], how='left', on='zhdh')

CPU times: user 33.3 s, sys: 158 ms, total: 33.5 s
Wall time: 26.1 s


In [8]:
%%time
# 远程控制app转账 - 每小时欺诈概率(前一小时，当前小时，后一小时的欺诈概率)
trans_info['hour'] = trans_info['jyrq_dayofyear'] * 24 + trans_info['jyrq_hour']

timeline = trans_info['hour'].unique()
ts_prob = dict()
for ts in timeline[1:-1]:
    li = [ts - 1, ts, ts + 1]
    tmp = trans_info[trans_info['hour'].isin(li)]['zhdh'].unique()
    ts_prob[ts] = train_set[train_set['zhdh'].isin(tmp)]['black_flag'].mean()
    
trans_info['hour_pro'] = trans_info['hour'].map(ts_prob)
hour_prob = trans_info.groupby('zhdh')['hour_pro'].mean().reset_index()

train_set = train_set.merge(hour_prob, how='left', on='zhdh')
test_set = test_set.merge(hour_prob, how='left', on='zhdh')

CPU times: user 17.8 s, sys: 157 ms, total: 17.9 s
Wall time: 18.1 s


In [9]:
%%time
# 账户存活时间短
trans_diff = trans_info.groupby('zhdh').agg({'jyrq': ['max', 'min']}).reset_index()
trans_diff.columns = ['zhdh', 'jyrq_max', 'jyrq_min']
trans_diff['trans_duration'] = (pd.to_datetime(trans_diff['jyrq_max']) - \
                                                pd.to_datetime(trans_diff['jyrq_min'])).dt.days

train_set = train_set.merge(trans_diff[['zhdh', 'trans_duration']], how='left', on='zhdh')
test_set = test_set.merge(trans_diff[['zhdh', 'trans_duration']], how='left', on='zhdh')

CPU times: user 1.05 s, sys: 25.1 ms, total: 1.07 s
Wall time: 1.07 s


In [10]:
%%time
# 天数少
day_count = trans_info.groupby('zhdh')['jyrq'].nunique().reset_index()
day_count.columns = ['zhdh', 'trans_day_count']

train_set = train_set.merge(day_count, how='left', on='zhdh')
test_set = test_set.merge(day_count, how='left', on='zhdh')

CPU times: user 216 ms, sys: 6.98 ms, total: 223 ms
Wall time: 222 ms


In [11]:
%%time
# 每天余额情况
day_rest_mean = trans_info.drop_duplicates(subset=['zhdh', 'jyrq'], keep='last').\
                            groupby('zhdh')['zhye'].mean().reset_index()
day_rest_mean.columns = ['zhdh', 'day_rest_mean'] 

train_set = train_set.merge(day_rest_mean, how='left', on='zhdh')
test_set = test_set.merge(day_rest_mean, how='left', on='zhdh')

CPU times: user 235 ms, sys: 39.3 ms, total: 274 ms
Wall time: 276 ms


In [12]:
# %%time
# # 趋势特征
# # 每周的转账次数情况、dfzh、dfhh等情况（算比例？）
# week_num = trans_info['jyrq_weekofyear'].unique()
# print(week_num)
# for idx in range(0, len(week_num), 2):
#     week_count = trans_info[trans_info['jyrq_weekofyear'].\
#                          isin([week_num[idx], week_num[idx + 1]])].groupby('zhdh').size().reset_index()
#     week_count.columns = ['zhdh', 'trans_count_of_week{}'.format(week_num[idx])]
    
#     train_set = train_set.merge(week_count, how='left', on='zhdh')
#     test_set = test_set.merge(week_count, how='left', on='zhdh')
    
#     week_sum = trans_info[trans_info['jyrq_weekofyear'].\
#                          isin([week_num[idx], week_num[idx + 1]])].groupby('zhdh')['jyje'].sum().reset_index()
#     week_sum.columns = ['zhdh', 'trans_sum_of_week{}'.format(week_num[idx])]
    
#     train_set = train_set.merge(week_sum, how='left', on='zhdh')
#     test_set = test_set.merge(week_sum, how='left', on='zhdh')


# for idx in range(2, len(week_num), 2):
#     train_set['count_ration_of_week_{}'.format(idx)] = train_set['trans_count_of_week{}'.format(week_num[idx])] / \
#                                                         train_set['trans_count_of_week{}'.format(week_num[idx - 2])]
#     test_set['count_ration_of_week_{}'.format(idx)] = test_set['trans_count_of_week{}'.format(week_num[idx])] / \
#                                                         test_set['trans_count_of_week{}'.format(week_num[idx - 2])]
    
#     train_set['sum_ration_of_week_{}'.format(idx)] = train_set['trans_sum_of_week{}'.format(week_num[idx])] / \
#                                                         train_set['trans_sum_of_week{}'.format(week_num[idx - 2])]
#     test_set['sum_ration_of_week_{}'.format(idx)] = test_set['trans_sum_of_week{}'.format(week_num[idx])] / \
#                                                         test_set['trans_sum_of_week{}'.format(week_num[idx - 2])]

# 每周特征
week_num = trans_info['jyrq_weekofyear'].unique()
agg_func = {
    'jyje': ['count', 'sum'],
    'dfzh': ['nunique'],
    'dfhh': ['nunique'],
    'jyqd': ['nunique']
}
new_col = ['{}_{}_of_week'.format(k, i) for k, v in agg_func.items() for i in v]

for week in week_num:
    
    agg_df = trans_info[trans_info['jyrq_weekofyear'] == week].groupby('zhdh').agg(agg_func).reset_index()
    agg_df.columns = ['zhdh'] + [i + str(week) for i in new_col]
    
    train_set = train_set.merge(agg_df, how='left', on='zhdh')
    test_set = test_set.merge(agg_df, how='left', on='zhdh')
    
    
for week in week_num[1:]:
    for col in new_col:
        # 差分特征
        train_set['diff_{}_{}_{}'.format(week, week - 1, col)] = \
                train_set[col + str(week)] - train_set[col + str(week - 1)]
        test_set['diff_{}_{}_{}'.format(week, week - 1, col)] = \
                test_set[col + str(week)] - test_set[col + str(week - 1)]
        
        # huanbi
        train_set['ration_{}_{}_{}'.format(week, week - 1, col)] = \
                train_set[col + str(week)] / train_set[col + str(week - 1)]
        test_set['ration_{}_{}_{}'.format(week, week - 1, col)] = \
                test_set[col + str(week)] / test_set[col + str(week - 1)]

In [13]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2023
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    
    oof = np.zeros(train_x.shape[0])
    predict = np.zeros(test_x.shape[0])

    cv_scores = []
    importance = []
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = \
                train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2020,
                'n_jobs':8
            }

            model = clf.train(params, train_matrix, 10000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=200, early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            print(list(sorted(zip(features, \
                                  model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.05,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 8
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=10000,
                              evals=watchlist, verbose_eval=1000, early_stopping_rounds=500)
            
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
                 
        if clf_name == "cat":
            
            model = clf(
                        n_estimators=10000,
                        random_seed=1024,
                        eval_metric='AUC',
                        learning_rate=0.05,
                        max_depth=5,
                        early_stopping_rounds=200,
                        metric_period=500,
                    )

            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      use_best_model=True,
                      verbose=1)
            
            val_pred  = model.predict_proba(val_x)[:,1]
            test_pred = model.predict_proba(test_x)[:,1]
            
        oof[valid_index] = val_pred
        predict += test_pred / kf.n_splits
        
        cv_scores.append(roc_auc_score(val_y, val_pred))
        print(cv_scores)
        importance += [model.get_fscore()]
       
    return oof, predict, importance

In [14]:
ts_fea = [i for i in train_set.columns if 'of_week' in i]
ts_cols = [i for i in basic_fea + [i for i in train_set.columns if 'of_week' in i] if i != 'black_flag']
ts_cols = [f for f in ts_cols if f not in ['zhdh','black_flag', 'khrq']]


xgb_oof, xgb_pred, xgb_importance = cv_model(xgb, train_set[ts_cols], train_set['black_flag'], test_set[ts_cols], 'xgb')

train_set['ts_prob'] = xgb_oof
test_set['ts_prob'] = xgb_pred

cols = [f for f in train_set.columns if f not in ['zhdh','black_flag', 'khrq'] + ts_fea]
xgb_oof, xgb_pred, xgb_importance = cv_model(xgb, train_set[cols], train_set['black_flag'], test_set[cols], 'xgb')


oof = xgb_oof
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4, 0.6, 0.01):
    preds = (oof.reshape((-1)) > threshold).astype('int')
    m = f1_score(train_set['black_flag'].values.reshape((-1)), preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m > best_score:
        best_score = m
        best_threshold = threshold
print(f'threshold - {best_threshold:.02f}, best f1 - {best_score}')

************************************ 1 ************************************
[0]	train-auc:0.90593	eval-auc:0.83537
[1000]	train-auc:0.99980	eval-auc:0.93361
[1058]	train-auc:0.99982	eval-auc:0.93398
[0.9339814814814814]
************************************ 2 ************************************
[0]	train-auc:0.90536	eval-auc:0.81873
[632]	train-auc:0.99955	eval-auc:0.87749
[0.9339814814814814, 0.8880028996013049]
************************************ 3 ************************************
[0]	train-auc:0.89243	eval-auc:0.84555
[778]	train-auc:0.99975	eval-auc:0.90172
[0.9339814814814814, 0.8880028996013049, 0.9085012285012285]
************************************ 4 ************************************
[0]	train-auc:0.86797	eval-auc:0.89943
[939]	train-auc:0.99974	eval-auc:0.95193
[0.9339814814814814, 0.8880028996013049, 0.9085012285012285, 0.9536364451618689]
************************************ 5 ************************************
[0]	train-auc:0.88828	eval-auc:0.82806
[679]	train-au

In [15]:
base = xgb_importance[0]

for sub in xgb_importance[1:]:
    for k, v in sub.items():
        base[k] = base.get(k, 0) + v

for k, v in sorted(base.items(), key=lambda x: x[1], reverse=True):
    print(k, v)

in_uniq 1022
ts_prob 719
hour_pro 681
day_rest_mean 629
word2vec_3 544
trans_duration 513
trans_day_count 502
word2vec_0 432
year 419
word2vec_1 385
年龄 331
trans_sum_ration 318
in_count 317
trans_count_ration 315
khjgdh 299
out_sum 260
day 254
in_sum 227
out_abs_uniq 224
word2vec_9 212
out_uniq 205
out_count 198
trans_out_count_5 167
month 149
word2vec_8 148
word2vec_4 148
word2vec_2 125
xb 101
word2vec_5 86
zero_count 85
trans_all_count_5 73
word2vec_6 71
word2vec_7 69
in_abs_uniq 64
age_level 58
trans_all_count_8 54
trans_out_count_3 29
trans_all_count_10 26
trans_all_count_3 16
trans_out_count_10 5
trans_out_count_8 2


In [16]:
# threshold - 0.44, best f1 - 0.7102132901597491
# threshold - 0.41, best f1 - 0.8506426890350612 - 0.75757575758 
# threshold - 0.51, best f1 - 0.8753058535667231 - 0.78381642512 
# threshold - 0.49, best f1 - 0.8732163783502427 - 0.78700361011 
# threshold - 0.47, best f1 - 0.8784692917942659 - 0.78871548619 
# threshold - 0.47, best f1 - 0.8787658362126447 -  0.79302045728 
# threshold - 0.45, best f1 - 0.8987062281443039 - 0.80263947211 hour_prob(nice呀)
# threshold - 0.46, best f1 - 0.9077434036533613 - ？还没尝试 应该nice  trans_duration
# threshold - 0.40, best f1 - 0.9117434071570816 - ？还没尝试 也应该nice  trans_day_count
# threshold - 0.40, best f1 - 0.9148403872302214 - 0.85645355850  day_rest_mean 
# threshold - 0.47, best f1 - 0.9049738098549112 - 0.86499402628  word2vec
# threshold - 0.51, best f1 - 0.9043998160944254 - 0.87617924528 ts_prob
# threshold - 0.49, best f1 - 0.9035974965953084 -  0.87773183698 ts_prob (修改特征)

In [17]:
pred = xgb_pred

train_set['oof'] = oof
# +++++++++++++++++++
best_threshold = 0.5
# +++++++++++++++++++
train_set['y_hat'] = train_set['oof'].apply(lambda x: 1 if x > best_threshold else 0)
train_set['error'] = train_set.apply(lambda x: abs(x['black_flag'] - x['oof']), axis=1)


test_set['black_flag'] = (pred.reshape((-1)) > best_threshold).astype(int)

test_set[['zhdh','black_flag']].to_csv('submission.csv', index=False)
test_set['black_flag'].mean(), train_set['black_flag'].mean()

(0.22208333333333333, 0.25)

In [18]:
head = train_set[train_set['error'] > 0.5].\
                sort_values(by='error')[['zhdh', 'error', 'oof', 'y_hat', 'black_flag']].head(30)


with pd.ExcelWriter('error.xlsx') as writer:  
    trans_info[trans_info['zhdh'].isin(train_set[train_set['black_flag'] == 1]['zhdh'].unique())].\
                                        to_excel(writer, sheet_name='black', index=None)
    static_info[static_info['zhdh'].isin(train_set[train_set['black_flag'] == 1]['zhdh'].unique())][keep_cols].\
                                        to_excel(writer, sheet_name='black_static', index=None)
#     trans_info[trans_info['zhdh'].isin(train_set[train_set['black_flag'] == 0]['zhdh'])].\
#                                         to_excel(writer, sheet_name='white', index=None)
#     static_info[static_info['zhdh'].isin(train_set[train_set['black_flag'] == 0]['zhdh'])][keep_cols].\
#                                         to_excel(writer, sheet_name='white_static', index=None)